In [2]:
import pandas as pd
import os #env variables
import requests #call the APIs
import json
from IPython.display import JSON

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [3]:
FOURSQUARE_KEY = os.getenv('FOURSQUARE_API_KEY')

In [9]:
city_row = df_bikes.iloc[0]
#first row

In [10]:
print(city_row)

Bike_Id        d0e8f4f1834b7b33a3faf8882f567ab8
City                 San Francisco Bay Area, CA
Bike_Lat                              37.849735
Bike_Lon                            -122.270582
Free_Bikes                                    6
Empty_Slots                                   9
Total_Bikes                                  15
Name: 0, dtype: object


In [12]:
city_row['Bike_Lat']

37.849735

In [6]:
#CSV file containing the latitudes and longitudes
file_path = r"..\data\df_bikes.csv"
df_bikes = pd.read_csv(file_path)

#Fsq data foreach lat/long
foursquare_data = []
for _, row in df_bikes.iterrows():
    foursquare_data.extend(getDetails(row["Bike_Id"],row["Bike_Lat"], row["Bike_Lon"]))

In [ ]:
def getDetails(id,lat,long):
    url = "https://api.foursquare.com/v3/places/search"
    params = {
        #"query": "convenience stores",
        "query": "bars",
        #Later with a loop could pull fsq data for EACH bike station
        "ll": f"{lat},{long}",  # Using latitude and longitude from the CityBikes df
        #"open_now": "true", #not needed
        "radius": "1000",
        "sort": "DISTANCE",
        'limit': 3
    }

    headers = {
        "Accept": "application/json",
        "Authorization": FOURSQUARE_KEY
    }

    # Make the API request
    response = requests.get(url, params=params, headers=headers)

    # Parse the JSON response
    data = response.json().get("results", [])

    # Extracting relevant data
    poi_data = []
    for result in data:
        name = result['name']
        #category = result['categories'][0]['plural_name']
        distance = result['distance']
        #address = result['location']['address']
        postcode = result['location']['postcode']
        #fsq_latitude = result['geocodes']['main']['latitude']
        #fsq longitude = result['geocodes']['main']['longitude']
        poi_data.append([id, lat, long, name, distance, postcode])
    
    
    return poi_data

In [ ]:
df_fsq = pd.DataFrame(foursquare_data, columns=["Bike_Id", "Bike_Lat", "Bike_Long", "name", "distance", "postcode"])

# Display or save/export the dataframe as needed
#print(df_fsq)
df_fsq.to_csv('..\data\df_fsq.csv', index=False)


Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [32]:
# Extracting relevant data
poi_data = []
for result in fsq_response['results']:
    #fsq_id = result['fsq_id'] #dont think i need this
    name = result['name']
    category = result['categories'][0]['plural_name']
    closed_bucket = result['closed_bucket']
    distance = result['distance']
    #address = result['location']['address']
    postcode = result['location']['postcode']
    latitude = result['geocodes']['main']['latitude']
    longitude = result['geocodes']['main']['longitude']
    region = result['location']['region']
    
    
    # Append data to the dict
    poi_data.append({
        #'fsq_id': fsq_id,
        'name': name,
        'category': category,
        'closed_bucket': closed_bucket,
        'distance': distance,
        #'address': address,
        'postcode': postcode,
        'latitude':latitude,
        'longitude':longitude,
        'region': region
    })

In [35]:
# Create a DataFrame from the list of dictionaries
df_foursquare = pd.DataFrame(poi_data)

In [ ]:
df_foursquare['region'] = "San Francisco Bay Area, CA"

In [ ]:
df_foursquare.head()

Put your parsed results into a DataFrame

In [38]:
df_foursquare.to_csv('..\data\df_foursquare.csv', index=False)

In [39]:
df_foursquare.shape

(1, 8)

In [41]:
df_foursquare.columns

Index(['name', 'category', 'closed_bucket', 'distance', 'postcode', 'latitude',
       'longitude', 'region'],
      dtype='object')

In [42]:
df_foursquare.dtypes

name              object
category          object
closed_bucket     object
distance           int64
postcode          object
latitude         float64
longitude        float64
region            object
dtype: object

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [4]:
#Old working code
YELP_API_KEY = os.getenv('YELP_API_KEY')
YELP_CLIENT_ID = os.getenv('YELP_CLIENT_ID')

yelp_url = "https://api.yelp.com/v3/businesses/search"

headers = {
    "Authorization": f"Bearer {YELP_API_KEY}",
}

params = {
    "term": "restaurant",
    "location": "San Francisco Bay Area",
    "radius": 1000,
    "limit": 3,
    #"sort_by": "distance", need to test this once ACCESS_LIMIT_RESTORED
    "client_id": YELP_CLIENT_ID,
}

yelp_response = requests.get(yelp_url, headers=headers, params=params)

if yelp_response.status_code == 200:
    #print(yelp_response.json())
    print("success status code 200")
else:
    print(f"Failed to get data. Status code: {yelp_response.status_code}, Error: {yelp_response.text}")


yelp_response = yelp_response.json()
JSON(yelp_response)

Failed to get data. Status code: 429, Error: {"error": {"code": "ACCESS_LIMIT_REACHED", "description": "You've reached the access limit for this client. See instructions for requesting a higher access limit at https://docs.developer.yelp.com/docs/fusion-rate-limiting"}}


<IPython.core.display.JSON object>

In [ ]:
#Using Lat long from EACH bike station

In [29]:
# Yelp data for each latitude and longitude pertaining to bikes
def getYelpDetails(id, lat, long):
    yelp_url = "https://api.yelp.com/v3/businesses/search"
    
    params = {
    "term": "restaurant",
    #"location": "San Francisco Bay Area",
    "radius": 1000,
    "limit": 3,
    "client_id": YELP_CLIENT_ID,
}

        
    url = f"{yelp_url}?latitude={lat}&longitude={long}"
    
    response = requests.get(url, params=params, headers=headers)
    #print(response.json())
    pois = response.json().get("businesses", [])
    
    extracted_data = []
    
    for poi in pois:
        name = poi["name"]
        review_count = poi["review_count"]
        categories = ', '.join([category["title"] for category in poi["categories"]])
        rating = poi["rating"]
        display_address = ', '.join(poi["location"]["display_address"])
        distance = poi["distance"]

        extracted_data.append([id,lat, long, name, review_count, categories, rating, display_address, distance])
    
    return extracted_data

# Read the Bike CSV file with the latitudes and longitudes
file_path = r"..\data\df_bikes.csv"
df_yelp = pd.read_csv(file_path)

# Yelp data for each set of latitude and longitude
all_yelps = []
for _, row in df_yelp.iterrows():  
    all_yelps.extend(getYelpDetails(row["Bike_Id"], row["Bike_Lat"], row["Bike_Lon"]))

In [31]:
df_all_yelp = pd.DataFrame(all_yelps, columns=["Bike_Id", "Bike_Lat", "Bike_Long", "name", "review_count", "categories", "rating", "display_address", "distance"])

# Display or save/export the dataframe as needed
#print(df_fsq)
df_all_yelp.to_csv('..\data\df_all_yelp.csv', index=False)


In [86]:
#testing 1 id
#print(response['businesses'][0]['id'])
#print(yelp_response['businesses'][0]['categories'][0]['alias'])

In [50]:

# Extracting relevant data
yelp_poi_data = []


for i in yelp_response['businesses']:
    city = i['location']['city']
    state = i['location']['state']
    name = i['name']
    review_count = i['review_count']
    rating = i['rating']
    latitude = i['coordinates']['latitude']
    longitude = i['coordinates']['longitude']
    alias = i['categories'][0]['alias']
      
   
      # Add more fields as needed
    
    # Append data and assign keys to the values
    yelp_poi_data.append({
        'city': city,
        'state': state,
        'name': name,
        'review_count': review_count,
        'rating': rating,
        'latitude': latitude,
        'longitude': longitude,
        'category': alias
        
 
    })


Put your parsed results into a DataFrame

In [51]:
df_yelp = pd.DataFrame(yelp_poi_data)

In [53]:
#del df_yelp['state']
df_yelp.head()

,city,state,name,review_count,rating,latitude,longitude,category
0,"San Francisco Bay Area, CA",CA,Cafe Noir,11,5.0,37.798756,-122.208274,burgers
1,"San Francisco Bay Area, CA",CA,Wawa Thai Food,243,4.5,37.797169,-122.204842,thai
2,"San Francisco Bay Area, CA",CA,Bombera,278,4.0,37.799530,-122.215200,mexican
3,"San Francisco Bay Area, CA",CA,The Saint Sandwich Shop,239,4.5,37.797250,-122.204370,sandwiches
4,"San Francisco Bay Area, CA",CA,Phnom Penh House Restaurant,506,4.5,37.791010,-122.197760,cambodian


In [54]:
df_yelp.to_csv('..\data\df_yelp.csv', index=False) #Export

In [74]:
df_yelp.columns

Index(['city', 'state', 'name', 'review_count', 'rating', 'latitude',
       'longitude', 'category'],
      dtype='object')

In [75]:
df_yelp.dtypes

city             object
state            object
name             object
review_count      int64
rating          float64
latitude        float64
longitude       float64
category         object
dtype: object

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

They both were pretty comparable. Seems like foursquare focuses more on location data and yelp has a more complete picture of the establishment and how it appeals to the masses. However, yelp also has some location info and establishment data relative to one another which is useful.

Get the top 10 restaurants according to their rating

In [64]:
top_10_resto = df_yelp.sort_values(by='rating', ascending=False)

In [56]:
top_10_resto.head(10)

,city,state,name,review_count,rating,latitude,longitude,category
0,"San Francisco Bay Area, CA",CA,Cafe Noir,11,5.0,37.798756,-122.208274,burgers
18,"San Francisco Bay Area, CA",CA,Maria’s Mexican Food,4,5.0,37.790938,-122.205074,tacos
3,"San Francisco Bay Area, CA",CA,The Saint Sandwich Shop,239,4.5,37.797250,-122.204370,sandwiches
4,"San Francisco Bay Area, CA",CA,Phnom Penh House Restaurant,506,4.5,37.791010,-122.197760,cambodian
5,"San Francisco Bay Area, CA",CA,Jo's Modern Thai,334,4.5,37.791680,-122.199110,thai
6,"San Francisco Bay Area, CA",CA,Boriqua Kitchen,128,4.5,37.804137,-122.207035,foodtrucks
1,"San Francisco Bay Area, CA",CA,Wawa Thai Food,243,4.5,37.797169,-122.204842,thai
2,"San Francisco Bay Area, CA",CA,Bombera,278,4.0,37.799530,-122.215200,mexican
7,"San Francisco Bay Area, CA",CA,Sequoia Diner,704,4.0,37.791851,-122.199130,diners
8,"San Francisco Bay Area, CA",CA,Grand Bakery,21,4.0,37.796780,-122.204500,bakeries


### Data Audit

In [81]:
null_values = df_yelp['rating'].isnull()

print(f"Number of null ratings: {null_values.sum()}")

Number of null ratings: 0


#### Data Clean city/state column

In [65]:
#df_yelp['city'] = df_yelp['city'] + df_yelp['state']

In [61]:
df_yelp.rename(columns={'city': 'location'}, inplace=True)

In [62]:
#df_yelp.drop(columns=['state'], inplace=True)

In [66]:
df_yelp.head()

,location,name,review_count,rating,latitude,longitude,category
0,"San Francisco Bay Area, CACA",Cafe Noir,11,5.0,37.798756,-122.208274,burgers
1,"San Francisco Bay Area, CACA",Wawa Thai Food,243,4.5,37.797169,-122.204842,thai
2,"San Francisco Bay Area, CACA",Bombera,278,4.0,37.799530,-122.215200,mexican
3,"San Francisco Bay Area, CACA",The Saint Sandwich Shop,239,4.5,37.797250,-122.204370,sandwiches
4,"San Francisco Bay Area, CACA",Phnom Penh House Restaurant,506,4.5,37.791010,-122.197760,cambodian


Import the city_bikes df

In [4]:
csv_file_path = '..\data\df_bikes.csv'

# Read the CSV file into a DataFrame
df_bikes = pd.read_csv(csv_file_path)

In [68]:
df_bikes.rename(columns={'location': 'City'}, inplace=True)

In [69]:
df_bikes.head()

,City,Latitude,Longitude,Free_Bikes,Empty_Slots,Total_Bikes
0,"San Francisco Bay Area, CA",37.849735,-122.270582,5,10,15
1,"San Francisco Bay Area, CA",37.336188,-121.889277,11,0,11
2,"San Francisco Bay Area, CA",37.322125,-121.881090,15,7,22
3,"San Francisco Bay Area, CA",37.323678,-121.874119,12,3,15
4,"San Francisco Bay Area, CA",37.325998,-121.877120,13,14,27


Union the 3 dataframes for ease and speed as I don't have time to sort out and clean all the dataframes:

### EDIT: Below concatanation produced insufficient results (NaN) values - Please see joining_data Notebook for revised Merge Function

In [25]:
#bikes_fsq_yelp_union = pd.concat([df_bikes, df_foursquare, df_yelp], ignore_index=True) #ignore_index=True new df has new index.

In [60]:
#bikes_fsq_yelp_union = pd.merge(df_bikes, df_foursquare, how='outer', on='name')

# Merging DataFrames failed
#merged_df = pd.merge(df_bikes, df_foursquare, left_on=['Latitude', 'Longitude'], right_on=['latitude', 'longitude'], how='inner')


In [62]:
#adding 3rd api failed
#merged_df = pd.merge(merged_df, df_yelp, left_on=['Latitude', 'Longitude'], right_on=['latitude', 'longitude'], how='inner')

In [71]:
#combined_df = pd.concat([df_bikes, df_foursquare, df_yelp], ignore_index=True)

In [72]:
#combined_df.tail()

,City,Latitude,Longitude,Free_Bikes,Empty_Slots,Total_Bikes,name,category,closed_bucket,distance,postcode,latitude,longitude,region,location,review_count,rating
570,NaN,NaN,NaN,NaN,NaN,NaN,Taqueria Ole Ole!,mexican,NaN,NaN,NaN,37.800653,-122.215375,NaN,"San Francisco Bay Area, CACA",10.0,3.5
571,NaN,NaN,NaN,NaN,NaN,NaN,World Ground Cafe,coffee,NaN,NaN,NaN,37.792170,-122.198810,NaN,"San Francisco Bay Area, CACA",281.0,3.5
572,NaN,NaN,NaN,NaN,NaN,NaN,Wingstop,chicken_wings,NaN,NaN,NaN,37.790437,-122.197862,NaN,"San Francisco Bay Area, CACA",74.0,2.0
573,NaN,NaN,NaN,NaN,NaN,NaN,Maria’s Mexican Food,tacos,NaN,NaN,NaN,37.790938,-122.205074,NaN,"San Francisco Bay Area, CACA",4.0,5.0
574,NaN,NaN,NaN,NaN,NaN,NaN,Domino's Pizza,pizza,NaN,NaN,NaN,37.792391,-122.199662,NaN,"San Francisco Bay Area, CACA",152.0,1.5


In [111]:
# Clean up the lat and lon column
# Remove brackets and commas from the 'Latitude' column
#combined_df['latitude'] = combined_df['latitude'].str.replace(r'[,\(\)]', '', regex=True)

# Convert the column to numeric
#combined_df['latitudeNew'] = combined_df['latitude'].str[1:-2]
#combined_df['latitudeNew'] = combined_df['latitude'].apply(lambda x: x.strip('()'))

Export the dataframe so that it can be accessed in the joining_data notebook

In [73]:
combined_df.to_csv('..\data\combined_apis.csv', index=False)

In [98]:
combined_df.dtypes

location          object
Latitude         float64
Longitude        float64
Free_Bikes       float64
Empty_Slots      float64
Total_Bikes      float64
name              object
category          object
closed_bucket     object
distance         float64
postcode          object
latitude         float64
longitude        float64
region            object
city              object
state             object
review_count     float64
rating           float64
dtype: object